In [ ]:
pip install transformers datasets


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 14.6 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("uqa/xlm-roberta-base-UQA-1.0")
model = AutoModelForQuestionAnswering.from_pretrained("uqa/xlm-roberta-base-UQA-1.0")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/716 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
pip install textattack

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 97.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 60.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 445.7/445.7 kB 31.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 776.5/776.5 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 53.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.3/143.3 kB 16.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import random
from transformers import AutoTokenizer, AutoModelForQuestionAnswering
from datasets import load_dataset
import torch
from textattack.attack_recipes import TextFoolerJin2019
from textattack.models.wrappers import HuggingFaceModelWrapper
from textattack.datasets import Dataset
from sklearn.metrics import f1_score
from textattack.shared import AttackedText


# Step 1: Load the model and tokenizer
tokenizer = AutoTokenizer.from_pretrained("uqa/xlm-roberta-base-UQA-1.0")
model = AutoModelForQuestionAnswering.from_pretrained("uqa/xlm-roberta-base-UQA-1.0")

# Step 2: Load the UQA dataset
dataset = load_dataset("uqa/UQA")

# Step 2.1: Randomly sample 5 examples from the validation dataset
random.seed(42)  # For reproducibility
random_indices = random.sample(range(len(dataset["validation"])), 5)
sampled_validation = dataset["validation"].select(random_indices)

# Step 3: Preprocess the data
def preprocess(example):
    inputs = tokenizer(
        example["question"], example["context"],
        max_length=512, truncation=True, padding="max_length", return_tensors="pt"
    )
    example["input_ids"] = inputs["input_ids"].squeeze(0).tolist()  # Convert tensors to list
    example["attention_mask"] = inputs["attention_mask"].squeeze(0).tolist()
    return example

tokenized_sampled_validation = sampled_validation.map(preprocess, batched=False)

# Step 4: Evaluate the model
def evaluate(example):
    inputs = {
        "input_ids": torch.tensor([example["input_ids"]]),
        "attention_mask": torch.tensor([example["attention_mask"]]),
    }
    with torch.no_grad():
        outputs = model(**inputs)
        start_logits = outputs.start_logits
        end_logits = outputs.end_logits

    # Get the most probable start and end positions
    start_idx = torch.argmax(start_logits, dim=1).item()
    end_idx = torch.argmax(end_logits, dim=1).item()

    # Decode the prediction
    predicted_answer = tokenizer.decode(
        inputs["input_ids"][0][start_idx: end_idx + 1], skip_special_tokens=True
    )
    example["predicted_answer"] = predicted_answer
    return example

results = tokenized_sampled_validation.map(evaluate)

# Step 5: Calculate metrics
def calculate_metrics(results):
    em = 0  # Exact Match
    f1 = 0  # F1-Score
    n = len(results)

    for example in results:
        correct_answer = example["answer"].strip()
        predicted_answer = example["predicted_answer"].strip()

        # Exact Match
        if correct_answer == predicted_answer:
            em += 1

        # F1-Score calculation
        correct_tokens = set(correct_answer.split())
        predicted_tokens = set(predicted_answer.split())
        common_tokens = correct_tokens & predicted_tokens

        precision = len(common_tokens) / len(predicted_tokens) if predicted_tokens else 0
        recall = len(common_tokens) / len(correct_tokens) if correct_tokens else 0
        f1 += 2 * precision * recall / (precision + recall) if (precision + recall) else 0

    em_score = em / n
    f1_score_avg = f1 / n
    return em_score, f1_score_avg

em_score, f1_score_avg = calculate_metrics(results)

print(f"Exact Match (EM): {em_score:.2f}")
print(f"F1 Score: {f1_score_avg:.2f}")

# Step 6: Perform adversarial attacks using TextAttack


for i, example in enumerate(results):
    print(f"Example {i+1}:")
    print(f"Context: {example['context']}\n")
    print(f"Question: {example['question']}\n")
    print(f"Correct Answer: {example['answer']}\n")
    print(f"Predicted Answer: {example['predicted_answer']}\n")
    print("="*80)


  0%|          | 0/10 [02:33<?, ?it/s]


Exact Match (EM): 0.40
F1 Score: 0.57
Example 1:
Context: 15 اگست ، 1971 کو ، ریاستہائے متحدہ امریکہ نے یکطرفہ طور پر بریٹن ووڈس معاہدے سے دستبردار ہوگیا۔ امریکہ نے گولڈ ایکسچینج اسٹینڈرڈ کو ترک کردیا جس کے ذریعہ ڈالر کی قیمت سونے کی قیمت سے منسلک کی گئی تھی اور دیگر تمام کرنسیوں کو ڈالر سے منسلک کیا گیا تھا ، جس کی قیمت کو تیرنے کے لئے چھوڑ دیا گیا تھا (مارکیٹ کی طلب کے مطابق بڑھنا اور گرنا) ۔ اس کے فورا بعد ، برطانیہ نے پیروی کی ، پاؤنڈ سٹرلنگ کو تیرنا۔ دیگر صنعتی ممالک نے اپنی متعلقہ کرنسیوں کے ساتھ پیروی کی۔ اس بات کی توقع کرتے ہوئے کہ کرنسی کی قیمتیں ایک وقت کے لئے غیر متوقع طور پر اتار چڑھاؤ کریں گی ، صنعتی ممالک نے اپنے ذخائر (اپنی رقم کی فراہمی میں توسیع کرکے) پہلے سے کہیں زیادہ مقدار میں بڑھائیں۔ اس کا نتیجہ ڈالر اور دیگر صنعتی ممالک کی کرنسیوں کی قدر میں کمی تھی۔ کیونکہ تیل کی قیمت ڈالر میں تھی ، تیل پروڈیوسروں کی حقیقی آمدنی کم ہوگئی۔ ستمبر 1971 میں ، اوپیک نے ایک مشترکہ اعلامیہ جاری کیا۔

Question: ریاستہائے متحدہ امریکہ نے بریٹن ووڈس معاہدے سے کب دستبرداری اختیار کی؟

Corr

In [ ]:
def preprocess_qa_with_context(example):
    input_text = f"Context: {example['context']} Q: {example['question']} A: {example['answer']}"
    return {"input_text": input_text, "label": example['answer']}

# Apply preprocessing to the train and validation splits
train_dataset = dataset['train'].map(preprocess_qa_with_context)
validation_dataset = dataset['validation'].map(preprocess_qa_with_context)

# Convert to TextAttack Dataset
from textattack.datasets import Dataset
textattack_train_dataset = Dataset(train_dataset)
textattack_validation_dataset = Dataset(validation_dataset)

# Now use TextAttack to attack the dataset
attacker = Attacker(attack, textattack_train_dataset)
attacker.attack_dataset()


Map:   0%|          | 0/124745 [00:00<?, ? examples/s]

Map:   0%|          | 0/16824 [00:00<?, ? examples/s]

{'id': ['56be85543aeaaa14008c9063', '56be85543aeaaa14008c9065', '56be85543aeaaa14008c9066', '56bf6b0f3aeaaa14008c9601', '56bf6b0f3aeaaa14008c9602'], 'title': ['بیونسے', 'بیونسے', 'بیونسے', 'بیونسے', 'بیونسے'], 'context': ["Beyoncé Giselle Knowles-Carter (/biː'j ⁇ nseɪ/ bee-YON-say) (پیدائش 4 ستمبر 1981) ایک امریکی گلوکارہ ، گانا لکھنے والی ، ریکارڈ پروڈیوسر اور اداکارہ ہیں۔ ہیوسٹن ، ٹیکساس میں پیدا ہوئی اور اس کی پرورش ہوئی ، اس نے بچپن میں مختلف گانے اور رقص کے مقابلوں میں پرفارم کیا ، اور 1990 کی دہائی کے آخر میں R&B گرل گروپ ڈسٹنی چائلڈ کے لیڈ گلوکار کی حیثیت سے شہرت حاصل کی۔ اس کے والد ، میتھیو نولز کے زیر انتظام ، یہ گروپ دنیا کے سب سے زیادہ فروخت ہونے والے گرل گروپس میں سے ایک بن گیا۔ ان کے وقفے نے بیونس کی پہلی البم ، خطرناک طور پر محبت میں (2003) کی رہائی دیکھی ، جس نے اسے دنیا بھر میں ایک سولو آرٹسٹ کے طور پر قائم کیا ، پانچ گریمی ایوارڈز حاصل کیے اور بل بورڈ ہاٹ 100 میں نمبر ون سنگلز میں محبت اور بچے پاگل لڑکے۔", "Beyoncé Giselle Knowles-Carter (/biː'j ⁇ nseɪ/ bee-YON-say) (پ


  0%|          | 0/10 [00:00<?, ?it/s]

KeyError: 1

In [ ]:
print(train_dataset[:5])

{'id': ['56be85543aeaaa14008c9063', '56be85543aeaaa14008c9065', '56be85543aeaaa14008c9066', '56bf6b0f3aeaaa14008c9601', '56bf6b0f3aeaaa14008c9602'], 'title': ['بیونسے', 'بیونسے', 'بیونسے', 'بیونسے', 'بیونسے'], 'context': ["Beyoncé Giselle Knowles-Carter (/biː'j ⁇ nseɪ/ bee-YON-say) (پیدائش 4 ستمبر 1981) ایک امریکی گلوکارہ ، گانا لکھنے والی ، ریکارڈ پروڈیوسر اور اداکارہ ہیں۔ ہیوسٹن ، ٹیکساس میں پیدا ہوئی اور اس کی پرورش ہوئی ، اس نے بچپن میں مختلف گانے اور رقص کے مقابلوں میں پرفارم کیا ، اور 1990 کی دہائی کے آخر میں R&B گرل گروپ ڈسٹنی چائلڈ کے لیڈ گلوکار کی حیثیت سے شہرت حاصل کی۔ اس کے والد ، میتھیو نولز کے زیر انتظام ، یہ گروپ دنیا کے سب سے زیادہ فروخت ہونے والے گرل گروپس میں سے ایک بن گیا۔ ان کے وقفے نے بیونس کی پہلی البم ، خطرناک طور پر محبت میں (2003) کی رہائی دیکھی ، جس نے اسے دنیا بھر میں ایک سولو آرٹسٹ کے طور پر قائم کیا ، پانچ گریمی ایوارڈز حاصل کیے اور بل بورڈ ہاٹ 100 میں نمبر ون سنگلز میں محبت اور بچے پاگل لڑکے۔", "Beyoncé Giselle Knowles-Carter (/biː'j ⁇ nseɪ/ bee-YON-say) (پ